# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
#imports
import matplotlib.pyplot as plt
import gmaps
import os
import requests
import pandas as pd
import numpy as np

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
#import in csv from previous activity
cities = pd.read_csv("cities.csv")

#delete strange column that spawned
del cities['Unnamed: 0']
cities.head()

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Caravelas,100,BR,overcast clouds,88,-17.7125,-39.2481,77.09,8.12
1,Phan Thiet,100,VN,overcast clouds,77,10.9333,108.1000,71.33,7.99
2,Sibolga,100,ID,overcast clouds,88,1.7427,98.7792,73.53,3.51
3,Vitim,100,RU,overcast clouds,98,59.4511,112.5578,5.83,4.09
4,Saint-Philippe,100,RE,light rain,87,-21.3585,55.7679,80.15,25.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#store humidty values for the heatlayer
humidity = cities["Humidity"].astype(float)
humidity_max = humidity.max()
print(humidity_max, humidity)

#create df for latlong of cities
city_latlng = cities[["Lat", "Lng"]]
city_latlng.head()

99.0 0      88.0
1      77.0
2      88.0
3      98.0
4      87.0
       ... 
548    84.0
549    77.0
550    77.0
551    56.0
552    82.0
Name: Humidity, Length: 553, dtype: float64


,Lat,Lng
0,-17.7125,-39.2481
1,10.9333,108.1000
2,1.7427,98.7792
3,59.4511,112.5578
4,-21.3585,55.7679


In [9]:
# create gmaps figure
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

#add heat map layer
heat_layer = gmaps.heatmap_layer(city_latlng, weights=humidity, max_intensity=humidity_max, dissipating=False, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#dataframe with specifications for ideal weather conditions 
clean_city = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0), :]

#remove rows with nan values
clean_city = clean_city.dropna(how='any')

#reset index and clean up columns
clean_city.reset_index(inplace=True)
del clean_city['index']

clean_city.head()

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Busselton,0,AU,clear sky,41,-33.6500,115.3333,73.96,2.98
1,Lafia,0,NG,clear sky,15,8.4833,8.5167,77.49,5.91
2,Collie,0,AU,clear sky,28,-33.3667,116.1500,77.00,3.53
3,South Venice,0,US,haze,87,27.0531,-82.4243,78.17,6.91
4,Cabo San Lucas,0,MX,clear sky,59,22.8909,-109.9124,74.61,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#empty varialble stored
hotel_list = []

for i in range(len(clean_city)):
    lat = clean_city.loc[i]['Lat']
    lng = clean_city.loc[i]['Lng']

    params = {"location": f"{lat},{lng}","radius": 5000,"types" : "hotel","key": g_key}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_list.append("")

# Append Hotel Name to the selected clean city data frame
clean_city["Hotel Name"] = hotel_list
hotel_df = clean_city.dropna(how='any')
hotel_df["Hotel Name"] = hotel_list
hotel_df.head() 

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Busselton,0,AU,clear sky,41,-33.6500,115.3333,73.96,2.98,Busselton
1,Lafia,0,NG,clear sky,15,8.4833,8.5167,77.49,5.91,Lafia
2,Collie,0,AU,clear sky,28,-33.3667,116.1500,77.00,3.53,Collie
3,South Venice,0,US,haze,87,27.0531,-82.4243,78.17,6.91,Venice
4,Cabo San Lucas,0,MX,clear sky,59,22.8909,-109.9124,74.61,8.05,Cabo San Lucas


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))